# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**
Данные геологоразведки трёх регионов находятся в файлах:
1. /datasets/geo_data_0.csv. Скачать датасет
2. /datasets/geo_data_1.csv. Скачать датасет
3. /datasets/geo_data_2.csv. Скачать датасет
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).




**Оглавление**: <a id= "toc"></a>
1. [Загрузка и подготовка данных](#21)
2. [Обучение и проверка модели](#2)
3. [Подготовка к рассчетку прибыли](#3)
4. [Расчет прибыли и рисков](#4)

## Загрузка и подготовка данных <a id=21></a> 

In [18]:
import pandas as pd
import numpy as np
from IPython.display import display

from sklearn.linear_model import LinearRegression
from sklearn.metrics import precision_recall_curve,roc_curve
from sklearn.metrics import r2_score

from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error, accuracy_score

#Grapthics
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

pd.set_option("precision", 2)

In [19]:
#загрузим данные
data_1 = pd.read_csv(r'/datasets/geo_data_0.csv')
data_2 = pd.read_csv(r'/datasets/geo_data_1.csv')
data_3 = pd.read_csv(r'/datasets/geo_data_2.csv')

In [20]:
#проверим загрузку данных
data_list = [data_1,data_2,data_3]
for data in data_list:
    display(data.head(2))
    print(data.info())    

,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,kBEdx,-15.00,-8.28,-5.88e-03,3.18
1,62mP7,14.27,-3.48,9.99e-01,26.95


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,fwXo0,-1.15,0.96,-0.83,27.76
1,WJtFt,0.26,0.27,-2.53,56.07


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


Данные считались, пропусков в данных нет.  В каждой таблице по 100 тысяч записей. Названия столбцов - одинаковые во всех 3 датасэтах. Менять типы мы не будем, т.к. задача исследовательская и пока не идет в продакшн.

In [21]:
#проверим на полные дубликаты
for data in data_list:
    print(data.duplicated().value_counts())    

False    100000
dtype: int64
False    100000
dtype: int64
False    100000
dtype: int64


Дубликаты строк в датасэтах отсутвуют. Поищем дубликаты по id.

In [22]:
#проверим id на  дубликаты
for data in data_list:
    print(data['id'].duplicated().value_counts())  
    duplicates = data[data['id'].duplicated()].id.tolist()
    #посмотрим глазами на дубликаты
    display(data[data['id'].isin(duplicates)].sort_values(by = 'id'))
    #display(data[].sort_values(by = 'id'))

False    99990
True        10
Name: id, dtype: int64


,id,f0,f1,f2,product
66136,74z30,1.08,-3.12e-01,6.99,127.64
64022,74z30,0.74,4.59e-01,5.15,140.77
51970,A5aEY,-0.18,9.36e-01,-2.09,33.02
3389,A5aEY,-0.04,1.57e-01,0.21,89.25
69163,AGS9W,-0.93,1.16e-01,-3.66,19.23
42529,AGS9W,1.45,-4.80e-01,0.68,126.37
931,HZww2,0.76,3.69e-01,1.86,30.68
7530,HZww2,1.06,-3.74e-01,10.43,158.83
63593,QcMuo,0.64,-4.73e-01,0.86,64.58
1949,QcMuo,0.51,-3.24e-01,-2.22,75.50


False    99996
True         4
Name: id, dtype: int64


,id,f0,f1,f2,product
5849,5ltQ6,-3.44,-12.30,2.00e+00,57.09
84461,5ltQ6,18.21,2.19,3.99e+00,107.81
1305,LHZR0,11.17,-1.95,3.00e+00,80.86
41906,LHZR0,-8.99,-4.29,2.01e+00,57.09
2721,bfPNe,-9.49,-5.46,4.01e+00,110.99
82178,bfPNe,-6.20,-4.82,3.00e+00,84.04
47591,wt4Uk,-9.09,-8.11,-2.31e-03,3.18
82873,wt4Uk,10.26,-9.38,4.99e+00,134.77


False    99996
True         4
Name: id, dtype: int64


,id,f0,f1,f2,product
45404,KUPhW,0.23,-1.70,4.99,11.72
55967,KUPhW,1.21,3.18,5.54,132.83
11449,VF7Jo,2.12,-0.86,5.75,181.72
49564,VF7Jo,-0.88,0.56,0.72,136.23
44378,Vcm5J,-1.23,-2.44,1.22,137.97
95090,Vcm5J,2.59,1.99,2.48,92.33
28039,xCHr8,1.63,0.37,-2.38,6.12
43233,xCHr8,-0.85,2.10,5.60,184.39


Не вижу "схожести" в занчений признаков и продукта для дубликатов. По индексу тоже нельзя выявить закономерность.

In [23]:
# по скольку мы не знаем откуда взялись дубликаты, то удалим их
# удалим так же столбец c id, для нашей задачи анализа он не информативен
for data in data_list:
    data.drop_duplicates(subset=['id'],inplace=True)
    print(data['id'].duplicated().value_counts())
    # удалим столбце с id
    data.drop(columns=['id'],axis=1, inplace=True)
    display(data.tail(1))

False    99990
Name: id, dtype: int64


,f0,f1,f2,product
99999,1.76,-0.27,5.72,149.63


False    99996
Name: id, dtype: int64


,f0,f1,f2,product
99999,-3.43,-7.79,-3.30e-03,3.18


False    99996
Name: id, dtype: int64


,f0,f1,f2,product
99999,-2.55,-2.03,6.09,102.78


In [24]:
# посмотрим на основные статистические показатели
def show_describe_features(feature):
    merge_data_1 = data_1[feature].describe()
    merge_data_2 = data_2[feature].describe()
    merge_data_3 = data_3[feature].describe()
    merge_data = pd.concat([merge_data_1,merge_data_2],axis=1)
    merge_data = pd.concat([merge_data,merge_data_3],axis=1)
    merge_data.columns = ['region1','region2','region3']
    display(merge_data)
    

In [25]:
show_describe_features('f0')

,region1,region2,region3
count,99990.00,99996.00,1.00e+05
mean,0.50,1.14,2.00e-03
std,0.87,8.97,1.73e+00
min,-1.41,-31.61,-8.76e+00
25%,-0.07,-6.30,-1.16e+00
50%,0.50,1.15,9.42e-03
75%,1.07,8.62,1.16e+00
max,2.36,29.42,7.24e+00


Рассмотрели основные статистические показатели для 3 регионов: 
- f0
 - Cреднее: 0.5, 1.15, 0.02. Нормированные средние значений данного признака  в датасэтах очень разнятся.
 - Медиана: 0.502405,1.153055,0.009424  Медиана в 1 и 2 не отличается от среднего. В 3 регионе данне немного перекоошены. 
 - СКО: 0.871844, 8.965815,1.732052.  Учитывя min/max с данным все впорядке
 - min/max: Сложно оценивать, т.к. изменена размерность данных
 - квантили: тоже оценивать нет смысла


In [26]:
show_describe_features('f1')

,region1,region2,region3
count,99990.00,99996.00,1.00e+05
mean,0.25,-4.80,-2.16e-03
std,0.50,5.12,1.73e+00
min,-0.85,-26.36,-7.08e+00
25%,-0.20,-8.27,-1.17e+00
50%,0.25,-4.81,-9.66e-03
75%,0.70,-1.33,1.16e+00
max,1.34,18.73,7.84e+00


 f1
 - Cреднее: 0.250141	-4.796608	-0.002159  Нормированные средние значений данного признака  в датасэтах очень разнятся. Во втором датасэте сильно выделяются данные.
 - Медиана:	0.250252	-4.813172	-0.009661 Медиана в 1 и 2 не отличается от среднего. В 3 регионе данне немного перекоошены. 
 - СКО: 	0.504430	5.119906	1.730397.  Учитывя min/max с данным все впорядке
 - min/max: Сложно оценивать, т.к. изменена размерность данных
 - квантили: тоже оценивать нет смысла

In [27]:
show_describe_features('f2')

,region1,region2,region3
count,99990.00,99996.00,99996.00
mean,2.50,2.49,2.50
std,3.25,1.70,3.47
min,-12.09,-0.02,-11.97
25%,0.29,1.00,0.13
50%,2.52,2.01,2.48
75%,4.72,4.00,4.86
max,16.00,5.02,16.74


- f2
 - Cреднее: 2.502629	2.494501	2.495084 Отличий по средним в данном признаке нету
 - Медиана: 2.515969	2.011475	2.484236.  Во втором регионе данные перекошены по данному признаку
 - СКО: 0.871844, 8.965815,1.732052.  Учитывя min/max с данным все впорядке
 - min/max: Сложно оценивать, т.к. изменена размерность данных
 - квантили: тоже оценивать нет смысла

In [28]:
show_describe_features('product')

,region1,region2,region3
count,99990.00,99996.00,99996.00
mean,92.50,68.82,95.00
std,44.29,45.94,44.75
min,0.00,0.00,0.00
25%,56.50,26.95,59.45
50%,91.85,57.09,94.93
75%,128.56,107.81,130.59
max,185.36,137.95,190.03


- product
 - Cреднее: 92.50	68.82	95.00 Среднее по 2 региону сильно отличается.
 - Медиана: 91.85	57.09	94.93 . Медианы не сильно уходят от средних есть небольшая скошенность влево.
 - СКО: 0.871844, 8.965815,1.732052.  Учитывя min/max с данным все впорядке
 - min/max: Сложно оценивать, т.к. изменена размерность данных
 - квантили: тоже оценивать нет смысла

In [29]:
# поделим выборки на тренировучную и валидационную в соотношении 75:25
def split_data(data):
    features = data.drop(['product'],axis=1)  # извлеките признаки
    target = data['product']

    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=890)
    
    #сбросим индексы у новых таблиц
    for df in features_train, features_valid, target_train, target_valid:
        df.reset_index(drop=True,inplace =True)
    print(features_train.shape, features_valid.shape, target_train.shape, target_valid.shape)
    return features_train, features_valid, target_train, target_valid

features_train_1, features_valid_1, target_train_1, target_valid_1 = split_data(data_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = split_data(data_2)
features_train_3, features_valid_3, target_train_3, target_valid_3 = split_data(data_3)

(74992, 3) (24998, 3) (74992,) (24998,)
(74997, 3) (24999, 3) (74997,) (24999,)
(74997, 3) (24999, 3) (74997,) (24999,)


В данном разделе мы:
- Посмотрели на данные, нашли немного дубликатов, удалили их.
- Выявили неинфомативный признак id. Удалили его.
- Посмотрели на статистики по данным Регионов, сравнили статистики по признакам. В целом распределение по данным близко к нормальному, но по 2 региону есть выбросы и средние по регионам отличаются друг от друга.
- поделили выборки на тестовые и валидационные

# Обучение и проверка модели  <a id=2></a>

In [30]:
#расчет универсальных метрик для алгоритма
def metrics_calculate(features_train,features_valid,target_train,target_valid,predicted_valid,model):
    model_metrics = []
    _r2_score = r2_score(target_valid,predicted_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse**0.5
    mae = mean_absolute_error(target_valid, predicted_valid) # найдите значение метрики MAE на обучающей выборке
    model_metrics.append([_r2_score,rmse,mae])
    return np.hstack((model_metrics))

#перечень метрик для инициализации массива и формирования заголовка к DF
def list_metrics():
    metrics = ['_r2_score','rmse','mae']
    return metrics

In [31]:
def model_LinearRegression(features_train,features_valid,target_train,target_valid):
    model = LinearRegression(n_jobs=120)
    model.fit(features_train,target_train)
    predicted_valid = model.predict(features_valid)


    model_metrics = metrics_calculate(
        features_train,features_valid,target_train,target_valid,predicted_valid,model
    )
    #print(model_metrics)
    return predicted_valid, model_metrics
    #print(model.coef_)


#напечатаем метрики модели и среднее количество тыс. баррелей/сутки
model_metrics = []
predicted_valid_1,metrics =  model_LinearRegression(features_train_1, features_valid_1, target_train_1, target_valid_1)
model_metrics.append(np.hstack(([metrics,predicted_valid_1.mean()])))

predicted_valid_2,metrics =  model_LinearRegression(features_train_2, features_valid_2, target_train_2, target_valid_2)
model_metrics.append(np.hstack(([metrics,predicted_valid_2.mean()])))

predicted_valid_3,metrics =  model_LinearRegression(features_train_3, features_valid_3, target_train_3, target_valid_3)
model_metrics.append(np.hstack(([metrics,predicted_valid_3.mean()])))

model_metrics = pd.DataFrame(model_metrics,columns =['accuracy','_r2_score','rmse','mae','predicted_mean']
,index= ['region_1','region_2','region_3'])
display(model_metrics.T)


,region_1,region_2,region_3
accuracy,0.28,1.00,0.20
_r2_score,0.28,1.00,0.20
rmse,37.64,0.89,39.95
mae,31.01,0.72,32.74
predicted_mean,92.48,69.03,95.02


Заметим, что во 2-м регионе что то не так с данными. Метрики показываеют высокое качество модели, т.к. MSE почти равно нулю, то модель начинает показывать идеальное значение. Что-то не так с данными в этом регионе. К сожалению мы не можем проверить пайплайн загрузки, и посмотреть, есть ли ошибка.

Минимальные значения метрик RMSE и MAE достигаются в первом регионе, в нем же и выше точность модели. В первом так же достигается максимальное значение метрики R2.
MAE нам говорит о том что среднее абсолютное отклонение модели равно 31 и 32.7, т.е. в нашей модели мы ошибаемся на 31 и 32.7 тыс. баррелей в сутки на каждом объекте при прогнозе добычи.

Метрика RMSE более чуствительна к большим значениям, чем метрика MAE т.е. у нее берется квадрат расстояния между истинным и предсказанным значением.

# Подготовка к расчёту прибыли <a id=3></a>

In [32]:
# введем исходные данные для расчета прибыли
budget = pow(10,10) # бютджет 10 миллиардов рублей
revenue_one_thousand_barrel = 450*pow(10,3)#доход с одной тыс. баррелей
quantity_oil_well = 25000 # количество нефтяных скважин в регионе

#Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.
brakeven = budget/quantity_oil_well 
print('Cумма, при которой скважина безубыточная равна {:.0f} тысяч рублей'.format(brakeven/pow(10,3))) 

#объем сырья для безубыточной разработки новой скважины
minimum_quantity_material= (budget/ revenue_one_thousand_barrel)/pow(10,3)
print('Минимальный объем сырья для безубыточной разработки новой скважины составляет {:.1f} тысячи баррелей'
      .format(minimum_quantity_material))# 22.2 тысячи баррелей

#Сравним полученный объём сырья со средним запасом в каждом регионе
show_describe_features('product')

Cумма, при которой скважина безубыточная равна 400 тысяч рублей
Минимальный объем сырья для безубыточной разработки новой скважины составляет 22.2 тысячи баррелей


,region1,region2,region3
count,99990.00,99996.00,99996.00
mean,92.50,68.82,95.00
std,44.29,45.94,44.75
min,0.00,0.00,0.00
25%,56.50,26.95,59.45
50%,91.85,57.09,94.93
75%,128.56,107.81,130.59
max,185.36,137.95,190.03


Сравнив со средними показаниями региона показания безубыточности в 22.2 тысячи баррелей против значения для каждого региона ``92.5068.82,95.00``  с одной скважины смотрятся оптимистично.  Так же мы посчитали сколько максимум денег мы можем потратить в среднем на одну скважину - это ``400 тысяч рублей``.

In [33]:
#функция расчета прибыли
def revenue(target_valid,predicted_valid):
   predicted_valid_200_index = predicted_valid.sort_values(ascending=False)[:200].index
   target_valid_200 = target_valid[target_valid.index.isin(predicted_valid_200_index)]
   target_valid_200_sum = target_valid_200.sum().round(2)
   income = target_valid_200_sum *revenue_one_thousand_barrel
   profit =  income -budget
   #risk = stats.percentileofscore(target_valid_200,minimum_quantity_material)
   #print("Риск, что нам попадутся безубыточноые скважины составил {:.2f}%".format(risk))
   return profit 
profit_1 = revenue(target_valid_1,pd.Series(predicted_valid_1))
profit_2 =revenue(target_valid_1,pd.Series(predicted_valid_2))
profit_3 =revenue(target_valid_3,pd.Series(predicted_valid_3))

# выведем данные
print('Валоваая прибыль для первого региона составила {:.2f} миллиарда:'.format(profit_1/pow(10,9)))
print('Валоваая прибыль для первого региона составила {:.2f} миллиарда:'.format(profit_2/pow(10,9)))
print('Валоваая прибыль для первого региона составила {:.2f} миллиарда:'.format(profit_3/pow(10,9)))

Валоваая прибыль для первого региона составила 3.12 миллиарда:
Валоваая прибыль для первого региона составила -1.40 миллиарда:
Валоваая прибыль для первого региона составила 2.51 миллиарда:


Средняя прибыль скважин по 3 региону оказалась обманчива. Валовая прибыль по топ 200 скважин там меньше чем в первом. Но это говорит о равномерности результата, что в 3 регионе скважины дают более равномерный доход. ##Но если посмотреть на СКО 

# Расчёт прибыли и рисков <a id =4></a>

Интервалы использовать нельзя, т.к. данные не поддаются нормальному распределению,  поэтому будем использовать метод квантилей. Воспользуемся техникой bootstrap


In [36]:
def bootstrap(target, probabilities):
    state = np.random.RandomState(890)

    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500,replace=True,random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample))# < напишите код здесь>)

    values = pd.Series(values)
    #простроим границы доверительного интервала 
    lower = values.quantile(0.025)/pow(10,9) 
    upper = values.quantile(0.975)/pow(10,9)
    mean = values.mean()/pow(10,9)
    # посчитаем риску убытков
    risk = stats.percentileofscore(values,minimum_quantity_material)
    print("Средняя валовая прибыль {:.2f} миллиарда".format(mean))
    print(" Границы 5% доверитлеьного интервала по валовой прибыли ({:.2f},{:.2f}) млрд.:".format( lower,upper))
    print("Риск убытков равен {:.1f}%".format(risk))
bootstrap_1 = bootstrap(target_valid_1,pd.Series(predicted_valid_1,index =target_valid_1.index ))
bootstrap_2 = bootstrap(target_valid_2,pd.Series(predicted_valid_2,index =target_valid_2.index))
bootstrap_3 = bootstrap(target_valid_3,pd.Series(predicted_valid_3,index =target_valid_3.index))
   

Средняя валовая прибыль 0.45 миллиарда
 Границы 5% доверитлеьного интервала по валовой прибыли (-0.02,0.94) млрд.:
Риск убытков равен 3.6%
Средняя валовая прибыль 0.47 миллиарда
 Границы 5% доверитлеьного интервала по валовой прибыли (0.08,0.87) млрд.:
Риск убытков равен 1.2%
Средняя валовая прибыль 0.33 миллиарда
 Границы 5% доверитлеьного интервала по валовой прибыли (-0.23,0.84) млрд.:
Риск убытков равен 11.0%


Перейдем к главной части нашего проекта - выбор региона для дальнейшей разведки. Т.к. мы выбирали 200 самых производительных скважин из нашей модели, то риск получения убытков получился небольшой ``3.6%,1.2%,11.0%`` соответственно. У 2 региона сымый низкий риск, самый небольшой интервал и самая высокая прибыль  ``0.47 миллиарда``.

Заметим, что валовая прибыль по всей выборке и валовая прибыль по ТОП-200 скважин сильно различается. Во втором регионе очень разбросаны данные и низкая средняя прибыль со скважины по сравнению с 3 регионом.

 Напомним что мы получили ТОП-200 скважин для нашей модели и заглянув в будущее взяли реальные значения по их добыче и посчитали валовую прибыль и риски. Оказалось что в 2 регионе валовая прибыль меньше и риски тоже. Будем советовать нефтянникам исследовать 2-й регион. 

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован